In [1]:
import gym
import os
import numpy as np
import tensorflow as tf

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
env = gym.make('LunarLander-v2')
obs = env.reset()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
n_inputs = 8
n_hidden = 16
n_outputs = 4

learning_rate = 0.001

initializer = tf.variance_scaling_initializer()

X = tf.placeholder(tf.float32, shape=[None, n_inputs])

hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.relu, kernel_initializer=initializer)
hidden = tf.layers.dense(hidden, n_hidden, activation=tf.nn.relu, kernel_initializer=initializer)
hidden = tf.layers.dense(hidden, n_hidden, activation=tf.nn.relu, kernel_initializer=initializer)
logits = tf.layers.dense(hidden, n_outputs)
outputs = tf.nn.softmax(logits) 
action = tf.squeeze(tf.multinomial(logits, num_samples=1), axis=-1)

y = tf.one_hot(action, n_outputs)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(cross_entropy)
gradients = [grad for grad, variable in grads_and_vars]
gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))
training_op = optimizer.apply_gradients(grads_and_vars_feed)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [4]:
def discount_rewards(rewards, discount_rate):
    discounted_rewards = np.zeros(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate
        discounted_rewards[step] = cumulative_rewards
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/reward_std for discounted_rewards in all_discounted_rewards]

In [ ]:
n_games_per_update = 1
n_max_steps = 1000
n_iterations = 5000
save_iterations = 10
discount_rate = 0.95
total_rewards =[]
with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        if len(total_rewards)>100:
            print("\rIteration: {}, Mean Reward: {}"
                  .format(iteration, (sum(total_rewards[-100:])/100)), end="")
        else:
            print("\rIteration: {}, Mean Reward: {}".format(iteration, 0), end="")
        all_rewards = []
        all_gradients = []
        total_rewards =  total_rewards[-120:]
        for game in range(n_games_per_update):
            current_rewards = []
            current_gradients = []
            obs = env.reset()
            game_reward = 0
            for step in range(n_max_steps):
                action_val, gradients_val = sess.run([action, gradients], feed_dict={X: obs.reshape(1, n_inputs)})
                
                obs, reward, done, info = env.step(action_val[0])
                game_reward+=reward
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                if done:
                    break
            total_rewards.append(game_reward)
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)

        all_rewards = discount_and_normalize_rewards(all_rewards, discount_rate=discount_rate)
        feed_dict = {}
        for var_index, gradient_placeholder in enumerate(gradient_placeholders):
            mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index]
                                      for game_index, rewards in enumerate(all_rewards)
                                          for step, reward in enumerate(rewards)], axis=0)
            feed_dict[gradient_placeholder] = mean_gradients
        sess.run(training_op, feed_dict=feed_dict)
        if sum(total_rewards[-100:])/100 > 199:
            print("training Compleate")
            saver.save(sess, "./my_policy_net_pg.ckpt")
            break
        if iteration%save_iterations==0:
            saver.save(sess, "./my_policy_net_pg.ckpt")
        

Iteration: 2423, Mean Reward: -103.41972206798756